Follow along: https://python.langchain.com/docs/use_cases/graph/constructing/

In [1]:
import os
import time

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_fireworks import ChatFireworks
from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# initialize env vars - load credentials
load_dotenv()

# initialize graph db:
graph = Neo4jGraph()

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=2000,
    )

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
    # alternative options: (these are just examples)
    # allowed_nodes=["Person", "Country", "Organization"],
    # allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)

Read reviews, prepare reviews:

In [2]:
def load_policies(
    file: str = "../data/clean/policy_chunks.csv",
    start_index: int = 0,
    limit: int = 100,
):
    # load dataframe:
    df = pd.read_csv(file)[start_index:start_index + limit]
    print(f"Loaded policies: {df.shape}")
    df["Content"] = df.apply(lambda row: f"{row['Header 1']}\n{row['Header 2']}\n{row['Concat']}".replace("\n\n", "\n").replace("\n\n", "\n"), axis=1)
    return df

Total policies: 440

Transform reviews to nodes and relationships:

In [3]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def convert_to_graph_documents(df: pd.DataFrame, page_content_column: str = "Content", llm_transformer: LLMGraphTransformer = None):
    # graph_documents = llm_transformer.convert_to_graph_documents(documents)
    loader = DataFrameLoader(df, page_content_column=page_content_column)
    documents = loader.load()
    # return  llm_transformer.convert_to_graph_documents(documents)
    graph_documents = []
    for doc in documents:
        try:
            # fireworks doesnt always return valid json
            graph_documents += llm_transformer.convert_to_graph_documents([doc])
        except:
            graph_documents += [None]
    return graph_documents

See example graphs:

In [4]:
# print(f"Nodes: {graph_documents[0].nodes}")
# print(f"Relationships: {graph_documents[0].relationships}")

Store dataframe with graph temporarily:

In [5]:
def store_graph_locally(
    df: pd.DataFrame,
    docs: list,
    path: str,
):
    df["graph"] = [gd.json() if gd is not None else None for gd in docs]
    df.to_csv(path, index=False)

Storing to graph database:

In [6]:
# graph.add_graph_documents(graph_documents)

Iteratively load these in chunks because, well, there's... a lot.

In [7]:
import time

# earlier:
# combination of fireworks and others

# together.ai: mixtral 
#start_index = 300
#limit = 500
#chunk_size = 10

start_index = 190
limit = 10000
chunk_size = 10


def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


graph_docs = []

policies_df = load_policies(start_index=start_index, limit=limit)
for idx, chunk_df in enumerate(chunker(policies_df, chunk_size)):
    idx_start = start_index + (idx * chunk_size)
    idx_end = start_index + ((idx + 1) * chunk_size)
    print(f"chunk {idx}: {idx_start} -> {idx_end}. {chunk_df.shape}")

    chunk_graph_docs = convert_to_graph_documents(chunk_df, llm_transformer=llm_transformer)
    graph_docs += chunk_graph_docs
    print(f"chunk {idx}: converted")

    store_graph_locally(chunk_df, chunk_graph_docs, f"../data/transformed/policies/policies-graph-{idx_start}-{idx_end}.csv")
    print(f"chunk {idx}: saved to csv")

    chunk_graph_docs = list(filter(None, chunk_graph_docs))
    graph.add_graph_documents(chunk_graph_docs)
    print(f"chunk {idx}: saved to neo4j")
    print(f"move index to: {idx_end}")

    time.sleep(1)

Loaded policies: (250, 4)
chunk 0: 190 -> 200. (10, 5)
chunk 0: converted
chunk 0: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 0: saved to neo4j
move index to: 200
chunk 1: 200 -> 210. (10, 5)
chunk 1: converted
chunk 1: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 1: saved to neo4j
move index to: 210
chunk 2: 210 -> 220. (10, 5)
chunk 2: converted
chunk 2: saved to csv


...

chunk 22: saved to neo4j
move index to: 420
chunk 23: 420 -> 430. (10, 5)
chunk 23: converted
chunk 23: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 23: saved to neo4j
move index to: 430
chunk 24: 430 -> 440. (10, 5)
chunk 24: converted
chunk 24: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 24: saved to neo4j
move index to: 440


GPT 3.5 vs GPT 4

In [29]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

policies_df = load_policies(start_index=start_index, limit=limit)
gd = convert_to_graph_documents(policies_df[0:1], llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)

Loaded policies: (440, 4)
20 [Node(id='Carry-On Bags', type='Item'), Node(id='Personal Item', type='Item'), Node(id='Diaper Bags', type='Item'), Node(id='Breast Pump', type='Item'), Node(id='Cooler Of Breast Milk', type='Item'), Node(id='Child Safety Seats', type='Item'), Node(id='Strollers', type='Item'), Node(id='Medical Or Mobility Devices', type='Item'), Node(id='Carry-On Requirements', type='Requirement'), Node(id='Soft-Sided Garment Bags', type='Item'), Node(id='Musical Instruments', type='Item'), Node(id='Special Items And Sports Equipment', type='Item'), Node(id='Pet Kennel Or Container', type='Item'), Node(id='Carry-On Pet Fee', type='Fee'), Node(id='Valet Bags', type='Item'), Node(id='Batteries', type='Item'), Node(id='Electronic Cigarettes', type='Item'), Node(id='Restricted Items', type='Item'), Node(id='Liquids', type='Item'), Node(id='Tsa', type='Organization')]
13 [Relationship(source=Node(id='Carry-On Bags', type='Item'), target=Node(id='Personal Item', type='Item'), ty

In [30]:
llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

policies_df = load_policies(start_index=start_index, limit=limit)
gd = convert_to_graph_documents(policies_df[0:1], llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)

Loaded policies: (440, 4)
14 [Node(id='Personal Item', type='Item'), Node(id='Carry-On Bag', type='Item'), Node(id='Diaper Bags', type='Item'), Node(id='Breast Pump', type='Item'), Node(id='Small, Soft-Sided Cooler Of Breast Milk', type='Item'), Node(id='Child Safety Seats', type='Item'), Node(id='Strollers', type='Item'), Node(id='Medical Or Mobility Devices', type='Item'), Node(id='Soft-Sided Garment Bags', type='Item'), Node(id='Musical Instruments', type='Item'), Node(id='Pet Kennel Or Container', type='Item'), Node(id='American Eagle', type='Airline'), Node(id='Embraer Erj-175', type='Aircraft'), Node(id='Tsa', type='Organization')]
10 [Relationship(source=Node(id='Diaper Bags', type='Item'), target=Node(id='Personal Item', type='Item'), type='EXCLUDED_FROM'), Relationship(source=Node(id='Breast Pump', type='Item'), target=Node(id='Personal Item', type='Item'), type='EXCLUDED_FROM'), Relationship(source=Node(id='Small, Soft-Sided Cooler Of Breast Milk', type='Item'), target=Node(i

In [37]:
document = """


1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.




1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.




1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
Jan 09, 2024 published  
Refunds of EVA/ UNI Air Ticket for Komatsu Routes due to Noto Peninsula Earthquakes in Japan","Ticket Changes/Refunds due to Flight Irregularities
Changes/Refunds of EVA/UNI Air Tickets affected by Aircraft Change
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.

1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.

1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
1. Eligibility  
Passengers holding BR (695) /B7(525) tickets with confirmed Premium Economy booking that being affected by aircraft change.        
2. Changes  
(1) Within ticket validity, passengers may choose any of the options below without reissue fee for ONE transaction, and the ticket should be reissued accordingly:   
A. Passengers may accept the re-accommodated economy booking class code of the same flight booked and refund the fare/tax difference reassessed based on the historical date; or  
B. Passengers may choose to rebook to the designated economy booking class code of the same route on other dates in accordance with the fare rule and pay/refund the fare/tax difference reassessed based on the historical date; or  
C. Passengers may choose to rebook to the same route, same premium economy booking class code as original ticket on other dates and pay/refund the fare/tax difference reassessed based on the historical date.  
D. Passengers may choose to rebook to business class of the same route on other dates in accordance with the fare rule and pay the fare/tax difference reassessed based on the historical date.  
(2) For tickets purchased from travel agents, please contact the original issuing agents for rebooking and ticket change.  
(3) If the flights revert to aircrafts with premium economy class service afterwards, passengers who have reissued tickets in compliance with this bulletin and request to change back to premium economy class with same flight/date may claim a refund or pay the fare/tax differences reassessed based on the historical fare with the original booking class code, and are entitled to waive change fee for one transaction.  
3.  Refunds  
(1) Refunds of the ticket and/or related ancillary services issued by means of Electronic Miscellaneous Document (EMD) may be made in accordance with provisions of involuntary refund that the refund service charge will not be imposed. Regarding Booking Service Charge (BSC), it is non-refundable for partial-used ticket. The unused BSC can only be refunded when the ticket is total-unused or consists of BSC imposed on a flight-coupon base.  
(2) Tickets purchased from travel agencies should refer to the original issuing travel agent.  
(3) Tickets purchased from EVA worldwide offices or EVA website, please contact EVA reservation and ticketing offices.   
4.  For tickets being changed with reissue fee waived but then voluntarily request for a change or refund afterwards, the service charge will not be waived.  
5. Award ticket and upgrade with mileages  
(1) Award tickets  
A. The service fee of USD$50 can be waived once. Any mileage difference will be collected or returned if the new itinerary requires more or less miles than the original one. Expired miles are exempt from collection.  
B. If an award ticket contains multiple segments in different route, zone or booking class, mileage deduction will be redeemed according to the higher cost one.  
C. If the flights revert to aircrafts with premium economy class service afterwards, passengers who request to change back to premium economy class, the mileage differences will be collected if the reissue ticket requires more miles than the original one. Expired mileage evaluation upon reissue can be exempt.  
The service fee of USD$50 for award ticket reissuance can be waived once for Green Card members. Diamond Card/Gold Card/Silver Card members are exempt from service fee as current policy.  
(2)  Upgrade awards  
Rebook the original ticket class, or change itinerary with new required mileage. Any mileage differences will be collected (returned).  
(3) Please contact EVA Air reservation and ticketing offices for rebooking and ticket change.  
(4) For the required mileage for Economy class to upgrade to Royal Laurel/Premium Laurel/Business Class, please refer to EVA website.  
6.  Group passengers shall contact the original issuing travel agency for changes/ refunds.  
7.  Free and Reduced fare tickets are not included in this handling guideline, e.g. ID/AD/DM...etc.  
Jan 09, 2024 published  
"""
document_df = pd.DataFrame([{"Content": document}])

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

policies_df = load_policies(start_index=start_index, limit=limit)
gd = convert_to_graph_documents(document_df, llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)

Loaded policies: (440, 4)
25 [Node(id='Passengers', type='Passenger'), Node(id='Tickets', type='Ticket'), Node(id='Premium Economy', type='Booking class'), Node(id='Aircraft Change', type='Event'), Node(id='Reissue Fee', type='Fee'), Node(id='Refund', type='Action'), Node(id='Rebooking', type='Action'), Node(id='Business Class', type='Booking class'), Node(id='Flight', type='Route'), Node(id='Fare/Tax Difference', type='Financial difference'), Node(id='Historical Date', type='Date'), Node(id='Travel Agents', type='Agent'), Node(id='Electronic Miscellaneous Document (Emd)', type='Document'), Node(id='Booking Service Charge (Bsc)', type='Fee'), Node(id='Unused Ticket', type='Ticket status'), Node(id='Eva Worldwide Offices', type='Office'), Node(id='Award Ticket', type='Ticket type'), Node(id='Upgrade', type='Action'), Node(id='Mileage', type='Measurement'), Node(id='Green Card Members', type='Membership level'), Node(id='Diamond Card Members', type='Membership level'), Node(id='Gold Card

In [40]:

llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0,
    max_tokens=2000,
)

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

policies_df = load_policies(start_index=start_index, limit=limit)
gd = convert_to_graph_documents(document_df, llm_transformer=llm_transformer)
print(len(gd[0].nodes), gd[0].nodes)
print(len(gd[0].relationships), gd[0].relationships)

Loaded policies: (440, 4)
20 [Node(id='Br (695) /B7(525) Tickets', type='Ticket'), Node(id='Premium Economy Booking', type='Booking'), Node(id='Aircraft Change', type='Event'), Node(id='Economy Booking Class Code', type='Booking'), Node(id='Business Class', type='Booking'), Node(id='Travel Agents', type='Organization'), Node(id='Premium Economy Class Service', type='Service'), Node(id='Electronic Miscellaneous Document (Emd)', type='Document'), Node(id='Booking Service Charge (Bsc)', type='Charge'), Node(id='Eva Worldwide Offices', type='Organization'), Node(id='Eva Website', type='Website'), Node(id='Award Tickets', type='Ticket'), Node(id='Upgrade Awards', type='Award'), Node(id='Eva Air Reservation And Ticketing Offices', type='Organization'), Node(id='Royal Laurel', type='Booking'), Node(id='Premium Laurel', type='Booking'), Node(id='Group Passengers', type='Group'), Node(id='Free And Reduced Fare Tickets', type='Ticket'), Node(id='Noto Peninsula Earthquakes', type='Event'), Node(i